Code die checkt of wij wav-files correct wegschrijven naar tf-records.


[link to original code](https://github.com/tensorflow/models/tree/master/research/audioset)

In [1]:
## Import packages
import os
import csv
import random
import numpy as np
import pandas as pd
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## De volgende bestanden komen uit het eerdere mapje 'models'. Nu heb ik ze in yt8m gezet.
## Deze zijn nodig om de embedding layer te maken
flags.DEFINE_string(
    'pca_params', 'models/vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', 'models/vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

flags.DEFINE_string(
    'yt_checkpoint', 'models/youtube_model.ckpt',
    'Path to the VGGish checkpoint file.')

In [3]:
# -2 voor silence
tfrecord_name = "test_embedding/-2.tfrecord"
wav_name = "test_embedding/youtube-2.wav"

In [4]:
target_video_id = [b'-2NVPmjojzE']

In [5]:
## Haal hier de example uit de tf-record (zoekend op video-id en dan handmatig checken op correcte seconden)

record_iterator = tf.python_io.tf_record_iterator(tfrecord_name)

## Met break: check alleen het eerste geluidsfragment uit de tf-record
embedding = None
    
for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)

    video_id = example.context.feature['video_id'].bytes_list.value
    
    #video_id = str(video_id)[3:-2]
    if(video_id == target_video_id):
        embedding = example.feature_lists.feature_list['audio_embedding']
        break;

In [6]:
len(embedding.feature[0].bytes_list.value[0])

128

In [7]:
# Eerste seconde
for byte in embedding.feature[1].bytes_list.value[0]:
    print(byte, end='')
    print(" ", end='')

102 46 81 138 66 105 0 144 155 156 131 105 146 238 106 78 0 235 145 153 0 54 100 227 225 136 80 60 174 152 72 255 189 141 255 112 158 141 0 86 233 0 255 53 173 73 208 178 108 66 161 239 118 201 132 174 229 121 130 15 137 124 255 92 0 57 177 210 123 0 255 137 99 130 135 140 47 152 255 0 0 167 242 255 121 0 109 242 245 0 16 255 0 99 149 0 234 0 0 0 211 255 158 239 87 149 110 153 255 85 255 255 215 86 69 216 82 246 46 0 165 0 108 188 86 80 68 0 

In [10]:
## Deze code is van inference.py gehaald.
## Op het einde wordt er een nieuwe sequence example gemaakt

## Function that takes examples from wav-file as input and returns a sequence example
def getSequenceExample(examples_batch, labels):
    
    # Prepare a postprocessor to munge the model embeddings.
    pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)

    with tf.Graph().as_default(), tf.Session() as sess:

        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch)

        postprocessed_batch = pproc.postprocess(embedding_batch)
        print(postprocessed_batch)


        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(embedding_batch, postprocessed_batch, seq_example)

In [11]:
## Maak eerst de examples
examples_batch = vggish_input.wavfile_to_examples(wav_name)
print("Examples shape: " + str(examples_batch.shape))

## Maak nu de embedding (met functie hierboven), pas PCA toe en maak er bytes van
embedding_batch, postprocessed_batch, seq_example = getSequenceExample(examples_batch, [161])

Examples shape: (10, 96, 64)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.22877741 0.17769393]
 [0.         0.         0.         ... 0.         0.22877741 0.17769393]]
[[165   0 143 ...   0  22 255]
 [165   0 142 ...   0  51 255]
 [165   0 142 ...   0  51 255]
 ...
 [166   0 147 ...   0   0 255]
 [175   8 146 ... 147  69 255]
 [175   8 146 ... 147  69 255]]


In [12]:
embedding_batch

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.22877741,
        0.17769393],
       [0.        , 0.        , 0.        , ..., 0.        , 0.22877741,
        0.17769393]], dtype=float32)

In [14]:
postprocessed_batch[0]

array([165,   0, 143, 116, 157,  61,  37,  95, 164, 203, 132,   0, 141,
       232, 117,  83,  15, 255, 194,  58,   6,  50, 126, 162, 255,  64,
       163, 130,  60, 193, 203, 255, 142,  74, 117, 189, 139, 255,  21,
        56, 115,   0, 207,  67, 109, 191, 241, 158,   9, 165, 175, 184,
       179, 145, 255, 204, 255,  57, 134, 164, 136, 213, 255,   7, 113,
       121,  93, 179, 172,  62, 201,   6, 215, 134, 119,  80,  77, 242,
       255,   0,   0, 158, 248, 255, 255, 208, 158, 255, 255,  97,   0,
       222,  87, 136, 255,  10, 237,   0,  78,   0,  95, 255, 245, 255,
         0, 102, 174,   0, 255, 202, 255, 255,   0,   7, 255, 255, 255,
       255,   0, 255,  51,   0,   0, 164, 237,   0,  22, 255], dtype=uint8)

In [15]:
byte_string_2 = seq_example.feature_lists.feature_list['audio_embedding'].feature[0].bytes_list.value[0]
byte_string_2

b'\xa5\x00\x8ft\x9d=%_\xa4\xcb\x84\x00\x8d\xe8uS\x0f\xff\xc2:\x062~\xa2\xff@\xa3\x82<\xc1\xcb\xff\x8eJu\xbd\x8b\xff\x158s\x00\xcfCm\xbf\xf1\x9e\t\xa5\xaf\xb8\xb3\x91\xff\xcc\xff9\x86\xa4\x88\xd5\xff\x07qy]\xb3\xac>\xc9\x06\xd7\x86wPM\xf2\xff\x00\x00\x9e\xf8\xff\xff\xd0\x9e\xff\xffa\x00\xdeW\x88\xff\n\xed\x00N\x00_\xff\xf5\xff\x00f\xae\x00\xff\xca\xff\xff\x00\x07\xff\xff\xff\xff\x00\xff3\x00\x00\xa4\xed\x00\x16\xff'

In [16]:
bs = np.fromstring(byte_string_2, dtype=np.uint8)
print(bs)

[165   0 143 116 157  61  37  95 164 203 132   0 141 232 117  83  15 255
 194  58   6  50 126 162 255  64 163 130  60 193 203 255 142  74 117 189
 139 255  21  56 115   0 207  67 109 191 241 158   9 165 175 184 179 145
 255 204 255  57 134 164 136 213 255   7 113 121  93 179 172  62 201   6
 215 134 119  80  77 242 255   0   0 158 248 255 255 208 158 255 255  97
   0 222  87 136 255  10 237   0  78   0  95 255 245 255   0 102 174   0
 255 202 255 255   0   7 255 255 255 255   0 255  51   0   0 164 237   0
  22 255]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


In [20]:
# Terug naar tf-record om te vergelijken
bs_2 = np.fromstring(embedding.feature[0].bytes_list.value[0], dtype=np.uint8)
bs_2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


array([115,  46,  90, 120,  40, 117,   0, 180, 161, 157, 132,  38, 165,
       246,  77, 106,   0, 255, 160, 133,   0,  25, 155, 242, 248,  72,
        72,  53, 191, 228,  83, 255, 132, 101, 248, 144,  86, 152,   0,
        61, 255,   0, 255,  40, 170,  87, 251, 224, 114,  63, 146, 235,
       154, 184, 159, 220, 255,  64, 133,   0, 140, 175, 255,  61,  20,
        94, 118, 202, 134,  22, 255, 121,  98, 111, 149, 103,  24, 237,
       255,   0,   0, 151, 193, 255, 139,  25, 142, 170, 200,  36,   0,
       255,   0, 110, 236,  50, 240,   0,   0,   0, 166, 255, 139, 255,
       110,  46, 139, 150, 255,   0, 255, 255, 194,   0, 161, 233, 238,
       211, 133,   0, 113,   0, 137, 255, 109,   1, 103,   0], dtype=uint8)